# Normalization with RUVg and ERCC spike-ins


In [ ]:
# Set up environment
options(warn = -1)
options(jupyter.plot_mimetypes = 'image/png')

suppressPackageStartupMessages(require(dplyr))
suppressPackageStartupMessages(require(ggplot2))
suppressPackageStartupMessages(require(tidyr))

suppressPackageStartupMessages(require(RUVSeq))
suppressPackageStartupMessages(require(EDASeq))

In [ ]:
source('src/load_datasets.r')

# Inspect the tables
cat('Metadata:')
head(meta, 10)

In Part 1, we created a tpm table summarized by gene. Here we'll repeat that, but also create a counts matrix summarized by gene, for differential expression analysis.

In [ ]:
# Stack matrix to tall table
counts_tall <- counts_mat %>%
    as.data.frame %>%
    mutate(gencode_tx = rownames(counts_mat)) %>%
    gather(Name, counts, starts_with('GS-'))

# Summarize by gene
gene_counts_tall <- counts_tall %>%
    inner_join(annot, by='gencode_tx') %>%
    group_by(hugo_symbol, Name) %>%
    summarize(counts = sum(counts))

# Pivot back to matrix
gene_counts <- gene_counts_tall %>%
    spread(Name, counts)
rownames(gene_counts) <- gene_counts$hugo_symbol
gene_counts$hugo_symbol <- NULL
gene_counts <- as.matrix(gene_counts)

cat('Tall table of transcript read counts (counts_tall):')
head(counts_tall, 3)

cat('Gene-level read counts matrix (gene_counts):')
gene_counts[1:3, 1:3]

cat('Matrix rows:')
nrow(gene_counts)
cat('Matrix columns:')
ncol(gene_counts)

## ERCC normalization effects

In [ ]:
get_spikes <- function(expr) {
    genes <- rownames(expr)[grep("^ERCC", rownames(expr), invert=TRUE)]
    spikes <- rownames(expr)[grep("^ERCC", rownames(expr))]
    return(list(genes=genes, spikes=spikes))
}                   

# Add ercc spike-ins to the gene counts
ercc <- counts_mat[get_spikes(counts_mat)$spikes,]

# Realign columns
ercc <- ercc[, colnames(gene_counts)]
gene_counts <- rbind(gene_counts, ercc)

cat('ERCC matrix:')
ercc[1:3, 1:3]
cat('Dimensions of new genes matrix:')
dim(gene_counts)

ERCC transcripts were quantified in the raw data and added to the bottom of the gene_counts matrix

In [ ]:
boxplot_by_treatment <- function(expr, title) {
    options(repr.plot.width=4, repr.plot.height=3)
    p <- expr %>%
        as.data.frame %>%
        gather('Name', 'counts') %>%
        inner_join(meta, by='Name') %>%
        filter(Concentration != 1) %>%
        ggplot
    p + geom_boxplot(aes(x = Timepoint, y=log10(counts), color=Treatment)) +
        theme(axis.text.x = element_text(angle = 90, hjust = 1)) +
        ggtitle(title)
}

boxplot_by_treatment(ercc, 'ERCC genes boxplots')
boxplot_by_treatment(gene_counts, 'Raw gene counts boxplots')
boxplot_by_treatment(sweep(gene_counts, 2, colSums(ercc), '/'), 
                     'Gene counts normalized by ERCC')

## RUV normalization
The RUV (remove unwanted variation) method uses a generalized linear model to find an appropriate normalization factor. This method allows us to quantify and remove any source of variation independent of the treatment conditions, including sample depth but also batch effects, sequencing lane, and unknown factors.

Following the RUV vignette, first we filter for genes with at least 5 reads in at least 2 samples.

In [ ]:
filter_genes <- function(expr, n_reads=5, n_samples=2) {
    filter <- apply(expr, 1, function(x) length(x[x > n_reads]) >= n_samples)
    return(expr[filter,])
}

calc_edger <- function(eset) {   
    design <- model.matrix(~Treatment, data=pData(eset))
    y <- DGEList(counts=counts(eset), group=meta$Treatment)
    y <- calcNormFactors(y, method="upperquartile")
    y <- estimateGLMCommonDisp(y, design)
    y <- estimateGLMTagwiseDisp(y, design)
    fit <- glmFit(y, design) 
    lrt <- glmLRT(fit, coef=2)
    top <- topTags(lrt, n=nrow(eset))$table
    return(top)
}
                    
run_ruvg <- function(expr) {
    filtered <- filter_genes(expr, n_samples=28)
    gene_types <- get_spikes(filtered)
    eset <- EDASeq::newSeqExpressionSet(as.matrix(filtered), phenoData=meta)
    ruv <- RUVSeq::RUVg(eset, gene_types$spikes, k=1)
    return(ruv)
}

In [ ]:
ruv <- run_ruvg(counts_mat)

dim(counts_mat)
dim(exprs(ruv))

In [ ]:
options(repr.plot.width=6, repr.plot.height=4)
plotRLE(counts_mat)
plotRLE(ruv)

## PCA plots

In [ ]:
plot_pca <- function(mat, plot_pairs=F) {
    options(repr.plot.width=6, repr.plot.height=4)

    pca_mat <- t(log10(mat + 1))
    
    # PCA
    pc_out <- prcomp(pca_mat)  
    pc_coords <- as.data.frame( pca_mat %*% pc_out$rotation[,1:4])
    pc_coords <- cbind(pc_coords, meta[rownames(pc_coords),])
 
    if (plot_pairs) {
        library(RColorBrewer)
        treatment_colors <- brewer.pal(3, "Set2")
        graphics::pairs(pc_coords[rownames(meta),1:4], col=treatment_colors[meta$Treatment], pch = as.integer(meta$Timepoint))
    } else {
        ggplot(pc_coords, aes(x=-PC1, y=PC2, color=Treatment, shape=Concentration, size=Timepoint)) + 
            geom_point(alpha=0.75)
    }
}

In [ ]:
n_samples <- ncol(counts_mat) - 1)  # num samples to consider for filtering threshold

plot_pca(counts_mat)
plot_pca(filter_genes(counts_mat, n_reads=5, n_samples=n_samples)
plot_pca(gene_counts)
plot_pca(filter_genes(gene_counts, n_reads=5, n_samples=n_samples))

In [ ]:
plot_pca(filter_genes(gene_counts, n_reads=5, n_samples=n_samples), plot_pairs=T)

In [ ]:
plot_density <- function(expr) {
    # Plot density
    options(repr.plot.width=6, repr.plot.height=5)

    p <- expr %>% 
        as.data.frame %>%
        gather('Name', 'counts') %>%
        ggplot
    p + geom_density(aes(x=counts, color=Name)) + scale_x_log10()
}

plot_density(counts_mat)
plot_density(gene_counts)

## Use RUVg with unchanging genes
If ERCC spike-ins are not available, or if they correlate with treatment, another gene set must be used for normalization. One alternative is to use all but the top 10,000 transcripts as the normalization factor.

In [ ]:
gene_counts <- gene_counts[, meta$Name]
filtered <- filter_genes(gene_counts, n_samples=28) # filter of 28 prevents zeros in one group
set_filtered <- EDASeq::newSeqExpressionSet(as.matrix(filtered), phenoData=meta)

In [ ]:
# Take all but top 10,0000 differentially expressed genes
top <- calc_edger(set_filtered)
empirical <- rownames(set_filtered)[which(!(rownames(set_filtered) %in% rownames(top)[1:10000]))]
plot(1:nrow(top),top$PValue)

In [ ]:
ruv <- RUVSeq::RUVg(set_filtered, empirical, k=1)

In [ ]:
save(ruv, file='data/ruv.rdata')